# DataSet Utils

This notebook includes python functions that would be used to create data set for model to process.

## Import libraries

- pytube for youtube utils
- moviepy.editor for clip operation
- os
- pygame
- csv/datetime/time for reading dataset table

In [1]:
from pytube import YouTube as ytb
import moviepy.editor as mpe
import os
import pygame
import csv
import datetime
import time
import gc

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def downloadClip(youtube_short_id, path):
    """
    Download video clips from youtube with provided path and store that in path.
    
    Args:
        youtube_short_id(string): short id of youtube clip
        path(string): download destination
        
    Returns:
        null
    
    """
    yt = ytb('https://www.youtube.com/' + youtube_short_id) \
        .streams \
        .filter(file_extension='mp4') \
        .first() \
        .download(path, youtube_short_id)
    return

In [3]:
def generate5sVedioClips(name, video_clip, start_time, end_time, des_path, index):
    """
    Cut the provide video clip into 5s clips. Store clips with name ends in 4 digits number e.g. "0001".
    
    Args:
        name(string):                          name of video clip
        vedio_clip(moviepy.video.videoClip):   video clip source
        start_time(int):                       start time (in sec) of the video where we want to start the cut
        end_time(int):                         end time (in sec)
        des_path(string):                      destination path to store generated clips
        index(int):                            number of generated clips
        
    Returns:
        null
    """
    if (end_time - start_time) < 5:
        print(name + " is not long enough, skip.")
        return index
    
    for i in range((end_time - start_time) // 5):
        start = start_time + i * 5
        sub_clip = video_clip.subclip(start, start + 5)
        sub_clip.write_videofile(des_path + "/" + format(index, "06d") + ".mp4", codec="mpeg4")
        index += 1
    
    return index

In [5]:
def parseAudioSetCsv(path):
    """
    The audioSet csv file contains information about each source video clip. The table would like this:
    
    short_id | start_time | end_time
    ---------|------------|---------
    id here  | 10         | 20
    ---------|------------|---------
    ...
    
    This function read the content from the given csv file and convert the time into seconds.
    
    Args:
        path(string):    source file path
        
    Returns:
        Data(int[][]):   2D array contains short_id, start_time and end_time
    """
    result = []
    
    with open(path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        line = 0
        for row in reader:
            line += 1
            if (line <= 3) :
                continue
            tmp = row
            tmp[1] = int(float(tmp[1]))
            tmp[2] = int(float(tmp[2]))
            result.append(tmp[:3])
    
    return result
    

In [6]:
def secondsToInt(timestamp):
    """
    Convert a timestamp from format "%M:%S" to seconds
    
    Args:
        timestamp(string):   timestamp like 00:10
        
    Returns:
        seconds(int):        second that timestamp represents
    """
    x = time.strptime(timestamp,'%M:%S')
    result = int(datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds())
    
    return result

In [7]:
def parseDataSetCsv(path):
    """
    The dataset csv file contains information about each source video clip. The table would like this:
    
    short_id | start_time | end_time
    ---------|------------|---------
    id here  | 00:10      | 00:20
    ---------|------------|---------
    ...
    
    This function read the content from the given csv file and convert the time into seconds.
    
    Args:
        path(string):    source file path
        
    Returns:
        Data(int[][]):   2D array contains short_id, start_time and end_time
    """
    result = []
    
    with open(path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        line = 0
        for row in reader:
            line += 1
            if (line == 1) :
                continue
            tmp = row
            tmp[1] = secondsToInt(tmp[1])
            tmp[2] = secondsToInt(tmp[2])
            result.append(tmp)
    
    return result

## Generate Data Set

This is the main function of this notebook, generateDataSet() would read in clip information from a csv file and download all videos and cut them into clips of 5s. The collections of 5s-clips would be used as data set for BGNet.

We use dataset from https://research.google.com/audioset/dataset/index.html 

In [8]:
def generateAudioSet(dataset_csv, videos_dir, clips_dir):
    """
    Generate dataset from dataset csv file. Download videos to one directory and cut 5s clips to clips directory
    
    Args:
        dataset_csv(string):     csv file contains dataset information
        videos_dir(string):      directory for videos
        clips_dir(string):       directory for 5s clips
    
    Returns:
        null
    """
    data = parseAudioSetCsv(dataset_csv)
    index = 0
    
    for d in data:
        print("----- executing ------")
        print(d)
        video_file = videos_dir + "/" + d[0] + ".mp4"
        try:
            downloadClip(d[0], videos_dir)
            clip = mpe.VideoFileClip(video_file, fps_source="fps")
            index = generate5sVedioClips(d[0], clip, d[1], d[2], clips_dir, index)
            print(d[0] + " succeed.")
        except:
            print(d[0] + " failed.")
        finally:
            if os.path.isfile(video_file):
                os.remove(video_file)
                print(d[0] + " deleted.")
        print("----- finished ------")
    
    return

We also have a self-collected dataset that contains dancing videos.

In [9]:
def generateDataSet(dataset_csv, videos_dir, clips_dir):
    """
    Generate dataset from dataset csv file. Download videos to one directory and cut 5s clips to clips directory
    
    Args:
        dataset_csv(string):     csv file contains dataset information
        videos_dir(string):      directory for videos
        clips_dir(string):       directory for 5s clips
    
    Returns:
        null
    """
    data = parseDataSetCsv(dataset_csv)
    index = 0
    
    for d in data:
        print("----- executing ------")
        print(d)
        video_file = videos_dir + "/" + d[0] + ".mp4"
        try:
            downloadClip(d[0], videos_dir)
            clip = mpe.VideoFileClip(video_file, fps_source="fps")
            index = generate5sVedioClips(d[0], clip, d[1], d[2], clips_dir, index)
            print(d[0] + " succeed.")
        except:
            print(d[0] + " failed.")
        finally:
            if os.path.isfile(video_file):
                os.remove(video_file)
                print(d[0] + " deleted.")
        print("----- finished ------")
    
    return

In [10]:
# generateDataSet("../resources/dance_dataset.csv", "../data/videos", "../data/5s_clips_dance")
# gc.collect()

----- executing ------
['p6filTvPb3U', 22, 202, '', '36', '']


chunk:  29%|██▉       | 32/111 [00:00<00:00, 182.94it/s, now=None]

Moviepy - Building video ../data/5s_clips_dance/000000.mp4.
MoviePy - Writing audio in %s


t:  30%|███       | 36/120 [00:00<00:00, 349.06it/s, now=None]     

MoviePy - Done.
Moviepy - Writing video ../data/5s_clips_dance/000000.mp4



...

Moviepy - Done !
Moviepy - video ready ../data/5s_clips_dance/000305.mp4
Moviepy - Building video ../data/5s_clips_dance/000306.mp4.
MoviePy - Writing audio in %s


t:   7%|▋         | 11/150 [00:00<00:01, 109.14it/s, now=None]     

MoviePy - Done.
Moviepy - Writing video ../data/5s_clips_dance/000306.mp4



Moviepy - Done !
Moviepy - video ready ../data/5s_clips_dance/000306.mp4
vt1w3iCX1q8 succeed.
vt1w3iCX1q8 deleted.
----- finished ------


583